# LLM API 비교
- 비용으로 1차적으로 추린 후에 2차적으로 성능 비교
## Gemini API
- 다른 LLM 라인업 대비 항상 좀 더 저렴한 가격에 제공

In [1]:
# 환경변수에 GOOGLE_API_KEY 추가 필요
# ~/.zshrc에 export GOOGLE_API_KEY='<API KEY>'
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel('gemini-1.5-flash')

/Users/hyeonjinho/Desktop/dev/Learned/llm/fastcampus-prompt-engineering-aillm-service/kakaotalk-summarization/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
response = model.generate_content("Hi!")
print(response.text)

Hi there! How can I help you today?



# Claude API
- OpenAI에 필적하는 성능
  - 2024년 6월 기준 가장 좋은 성능의 모델 = Claude 3.5 Sonnet
- 현재 기준 가장 비싼 모델인 Claude 3 Opus 보유

In [3]:
# 환경변수에 ANTHROPIC_API_KEY 추가 필요
# ~/.zshrc에 export ANTHROPIC_API_KEY='<API KEY>'
import os

import anthropic

client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0.0,
    system="Respond only in Yoda-speak.",
    messages=[
        {"role": "user", "content": "Hi!"}
    ]
)

print(message.content[0].text)

Greetings, young one. How may I assist you today?


# 동일한 Prompt에 대해 Input / Output 계산
- 비용 계산을 위해서는 Tokens 수를 알아야 함

In [5]:
from eval import get_eval_data

conversations = get_eval_data()

In [6]:
print(conversations[0])


P01: 맞다요즘 싸이월드 새로 생겼다매
P02: 나 싸이월드 복구하고 싶다
P01: 웅 키키귀여우뉴사진
P02: 진짜 추억돋네
P01: 짱많아 옛날 사진들
P01: 맨날 퍼가요~ 이거 했자나
P02: 맞아 싸이월드 미니미
P01: 추억 돋아서 너무조아
P02: 꾸미는거
P01: 마자
P02: 재밌었는데
P01: 귀여워 도토리충전
P02: 네이트온도 하고
P02: 도토리도 환불해준대자나
P01: 마자 키키개웃겨
P01: 요즘 사회적으로 멀 자꾸하나바
P02: 대박이야 진짜
P02: 그니깐 키키
P01: 아니 나오늘 엄마가 뭐 신청해달라그래서
P02: 웅
P01: 지원금 ? 신청함
P01: 진짜 요즘 지원금 엄청 많이 받았어
P02: 와 대박
P01: 뭔지는 잘 모르는데 이것저것 지원해주는거 많아서 좋은듯
P02: 다행이다
P01: 국가제도가 진짜 괜차나
P01: 학교에서도
P02: 두분다
P01: 맨날 장학금 해준다고
P02: 일을 못하는
P01: 이것저것 지원해주자나
P02: 상황이셔서 더
P01: 마자ㅠㅠ
P02: 도움주시고
P01: 나 그래서 진짜 그런거로
P02: 다행이다그래도
P01: 돈 마니받앗어
P02: 와대박


Gemini Pro & Flash

- 506 / 116 / 622

In [7]:
prompt = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:

{conversations[0]}"""

model = genai.GenerativeModel('gemini-1.5-flash-001')
response = model.generate_content(prompt)
print(response.text)
response.usage_metadata

두 사용자는 싸이월드 부활과 추억에 대한 이야기를 나누며 옛날 사진과 꾸미기, 도토리 등을 회상합니다. 또한, 요즘 사회적으로 지원금과 장학금 등 다양한 도움을 받을 수 있는 점에 대해 공감하며 긍정적인 반응을 보입니다. 특히 P01은 엄마를 대신해 지원금을 신청했고, 학교에서도 장학금을 받아 돈을 많이 벌었다는 사실을 P02에게 자랑스럽게 이야기합니다. 



prompt_token_count: 506
candidates_token_count: 136
total_token_count: 642

모델 호출해보기 전에 Prompt가 몇 토큰인 지 미리 알 수 있음

In [8]:
# 실제로는 여기에 token 1씩 더 추가됨
model.count_tokens(prompt)

total_tokens: 505

동일한 Prompt에 대해 Gemini Pro와 Gemini Flash 둘 다 506 입력 토큰 -> 동일한 Tokenizer를 사용하는 것을 알 수 있음

- 더 많은 단어를 익힌 (보통 더 큰) 모델의 경우 다른 Tokenizer를 써서 입력 토큰 수가 달라질 수 있음

In [9]:
prompt = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:

{conversations[0]}"""

model = genai.GenerativeModel('gemini-1.5-pro-001')
response = model.generate_content(prompt)
print(response.text)
response.usage_metadata

두 사람은 과거 싸이월드를 하던 시절을 회상하며 추억에 잠깁니다. 그러다가 최근 정부와 학교에서 지원금을 많이 받았다는 주제로 이야기가 넘어가며 만족감을 표합니다. 특히 P01은 다양한 지원금 덕분에 돈을 많이 받았다고 자랑합니다. 



prompt_token_count: 506
candidates_token_count: 92
total_token_count: 598

Anthropic Claude

- 637 / 143 / 780

In [15]:
import os

import anthropic

client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(message.content[0].text)
message.usage

1. 두 사람은 과거 싸이월드 사용 경험을 회상하며 추억을 공유하고 있다.
2. 최근 정부와 학교에서 제공하는 다양한 지원금 혜택에 대해 이야기하고 있다.
3. 경제적 어려움을 겪고 있는 상황에서 이러한 지원이 도움이 되고 있다고 말하고 있다.


Usage(input_tokens=637, output_tokens=143, cache_creation_input_tokens=0, cache_read_input_tokens=0)

In [18]:
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(message.content[0].text)
message.usage

이 대화는 싸이월드의 재출시와 그에 대한 추억을 나누는 것으로 시작합니다. 그 후 화제가 최근의 사회적 지원금과 장학금 등 다양한 국가 지원 제도로 전환됩니다. 대화 참여자들은 이러한 지원 제도에 대해 긍정적인 반응을 보이며, 특히 어려운 상황에 있는 사람들에게 도움이 된다고 평가합니다.


Usage(input_tokens=637, output_tokens=168, cache_creation_input_tokens=0, cache_read_input_tokens=0)

In [19]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(message.content[0].text)
message.usage

1. 싸이월드가 새로 생겨서 옛날 추억을 되새기며 즐거워하는 두 사람의 대화이다.
2. 요즘 사회적으로 여러 지원금과 장학금 등 국가 제도의 도움을 많이 받고 있다는 내용이 언급되었다.
3. 두 사람 모두 일을 못하는 상황이지만 국가의 지원으로 도움을 받고 있어 다행이라고 생각하고 있다.


Usage(input_tokens=637, output_tokens=170, cache_creation_input_tokens=0, cache_read_input_tokens=0)

In [20]:
from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)
completion.usage

두 사용자는 싸이월드를 통해 추억을 공유하고, 최근 지원금을 받아 도움을 받았다는 이야기를 나누며 국가제도에 대해 이야기하고 있다.


CompletionUsage(completion_tokens=73, prompt_tokens=622, total_tokens=695, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0})

In [21]:
from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-4-turbo-2024-04-09',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)
completion.usage

사용자들은 최근 재개된 싸이월드에 대해 이야기하며, 옛날 사진과 미니미 꾸미기 등 추억을 회상합니다. 또한, 사용자들은 정부 지원금과 장학금 등 다양한 지원 제도에 대해 긍정적인 의견을 나눕니다. 이 대화에서는 싸이월드의 복구와 추억, 그리고 사회적 지원 제도에 대한 만족감이 주된 주제로 다뤄집니다.


CompletionUsage(completion_tokens=158, prompt_tokens=622, total_tokens=780, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0})

In [22]:
from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-4o-2024-05-13',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)
completion.usage

사용자들은 싸이월드의 복구와 관련된 추억을 이야기하며, 옛날 사진과 미니미 꾸미기, 도토리 충전 등의 재미를 회상하고 있습니다. 또한, 최근 정부의 다양한 지원금 제도에 대해 이야기하며, 이를 통해 많은 도움을 받고 있다는 점을 공유하고 있습니다. 대화는 싸이월드와 지원금 제도에 대한 긍정적인 경험을 나누는 내용으로 구성되어 있습니다.


CompletionUsage(completion_tokens=102, prompt_tokens=441, total_tokens=543, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0})

Input Tokens 수 정리 (Prompt 고정)

- Gemini 506
- Claude 637
- GPT 622
- GPT-4o 441
  - Output은 어느 정도 조정이 가능해서 Input만 비교

한글 효율화 측면에서는 GPT-4o > Gemini > GPT-3.5/4 >= Claude 순

- 다른 Prompt에서는 조금씩 다를 수 있으나, 경향이 크게 달라지지 않을 것으로 예상